In [56]:
! pip install --quiet python-dotenv openai tabulate
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [2]:
import evadb
import shutil
import pandas as pd
import os
from dotenv import load_dotenv

In [3]:
shutil.rmtree('./evadb_data')

try:
    cursor = evadb.connect().cursor()
    cursor.query("""
        CREATE DATABASE mydb WITH ENGINE = 'sqlite', PARAMETERS = {
        "database": ":memory:"
    };""").execute()
except:
    pass

In [29]:


# drop all tables
cursor.query("""
    DROP TABLE IF EXISTS tasks;
""").execute()
# cursor.query("""
#     DROP TABLE IF EXISTS members;
# """).df()

# Create tables
cursor.query("""
    CREATE TABLE tasks (
        id INTEGER,
        name TEXT(30),
        description	TEXT(100)
    );
""").execute()


In [22]:
# list tables
x = cursor.query("""
use mydb {
   SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%'
};
""").df()

print(x)

Empty DataFrame
Columns: []
Index: []


In [50]:
# create tasks

# y = cursor.query("""
#  INSERT INTO tasks (id, name, description) VALUES (3, 'test name 2', 'test description');
#     """).df()
# print(y)


# drop all data in tasks

# read json using pd
json_data = pd.read_json('issues.json')
print(json_data.head())

# insert each row into table
for index, row in json_data.iterrows():
    # sanitize
    desc = row['description'].replace("'", "")
    desc = desc.replace(';', '')
    desc = desc.replace(',', '')

    cursor.query("""
    INSERT INTO tasks (id, name, description) VALUES ({}, '{}', '{}');
    """.format(index, row['contributor'], desc)).execute()

                                         description contributor
0  For now, if open's pr adds or changes long int...     xzdandy
1  Check if the query plan uses VectorIndexScan f...    jiashenC
2  I'm planning to add a new CONCAT operator to E...  pchunduri6
3  When training models, tuning the parameters ar...     xzdandy
4  Right now, the binder_context is not getting u...   gaurav274


In [ ]:
tasks = cursor.table('tasks').select("*").df()
print(tasks.head())

   tasks._row_id  tasks.id  tasks.name  \
0              1         0     xzdandy   
1              2         1    jiashenC   
2              3         2  pchunduri6   

                                   tasks.description  
0  For now if opens pr adds or changes long integ...  
1  Check if the query plan uses VectorIndexScan f...  
2  Im planning to add a new CONCAT operator to Ev...  


c:\Users\Rambe\anaconda3\envs\evadb\Lib\site-packages\evadb\models\storage\batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(


In [11]:
# print(os.environ['OPENAI_KEY'])

cursor.query("""
CREATE FUNCTION IF NOT EXISTS OpenAIChatCompletion
IMPL 'evadb/functions/openai_chat_completion_function.py'
MODEL 'gpt-3.5-turbo'
""").execute()

In [63]:
sample_issue = "When training models, tuning the parameters are common. Adding a CREATE OR REPLACE to simplify the query, so users do not need to run a DROP query every time."
context = f"""
Give a similarity score between this and the sample task below. Here are all of the tasks as context:
All tasks:
{tasks.to_markdown()}
"""
# context = f"""
# Which of the following tasks is this most similar to?
# All tasks:
# {tasks.to_markdown()}
# """


# query = f"Give only a value between 0 and 1 representing the similarity between the given description and the following sample issue: {sample_issue}"
prompt = "Give only a value between 0 and 1 for similarity and limit your response to 3 words."
# prompt = "Give only the tasks.id of the most similar task. Limit your response to 3 words."
res = cursor.table("tasks").select(
    f"ChatGPT('{context}', description, '{prompt}')"
)
responses = res.df()



print(responses)

c:\Users\Rambe\anaconda3\envs\evadb\Lib\site-packages\evadb\models\storage\batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(


   chatgpt.response
0               0.9
1               0.2
2               0.5
3               0.9
4               0.2
5               0.5
6               0.5
7               0.2
8               0.2
9               0.2
10              0.2
11              0.2
12              0.2
13              0.5
14              0.2
15              0.2
16              0.2
17              0.2
18              0.2
19              0.2
20              0.2
21              0.2
22              0.2
23              0.2
24              0.2


c:\Users\Rambe\anaconda3\envs\evadb\Lib\site-packages\evadb\models\storage\batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(


In [64]:
# print all of the responses in responses['chatgpt.response']
for index, row in responses.iterrows():
    print(row['chatgpt.response'])

0.9
0.2
0.5
0.9
0.2
0.5
0.5
0.2
0.2
0.2
0.2
0.2
0.2
0.5
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
0.2
